# External Inconsistencies

In [30]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from IPython.display import display
import pyomo.environ as pyo
from pyomo.opt import SolverFactory


In [67]:
countries = {'Austria': 'AT', 'Belgium': 'BE',  'Bulgaria': 'BG', 'Switzerland': 'CH', 'Czech Republic': 'CZ',  'Germany': 'DE', 'Denmark': 'DK', 'Estonia': 'EE', 'Spain': 'ES', 'Finland': 'FI', 'France': 'FR',  'Greece': 'GR', 'Hungary': 'HU',
             'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Latvia': 'LV', 'Montenegro': 'ME', 'Netherlands': 'NL', 'Norway': 'NO', 'Poland': 'PL', 'Portugal': 'PT', 'Serbia': 'RS', 'Sweden': 'SE', 'Slovenia': 'SI', 'Slovakia': 'SK', 'United Kingdom': 'GB'}
abbr_list = list(countries.values())
abbr_list = [country for country in abbr_list if country != 'CH']

In [69]:
generation_types = {'Coal': ['Fossil Hard coal', 'Fossil Brown coal/Lignite'],
                    'Biomass': ['Biomass'],
                    'Fossil Gas': ['Fossil Gas', 'Fossil Coal-derived gas'],
                    'Fossil Oil': ['Fossil Oil'],
                    'Fossil Oil shale': ['Fossil Oil shale'],
                    'Fossil Peat': ['Fossil Peat'],
                    'Waste': ['Waste'],
                    'Geothermal': ['Geothermal'],
                    'Hydro': ['Hydro Pumped Storage', 'Hydro Run-of-river and poundage', 'Hydro Water Reservoir'],
                    'Nuclear': ['Nuclear'],
                    'Other renewable': ['Other renewable'],
                    'Solar': ['Solar'],
                    'Other': ['Other'],
                    'Wind': ['Wind Offshore', 'Wind Onshore']}


In [74]:
df = pd.read_csv(f'../Data Sources/ENTSO-E/2018/Generation/Generation Capacity/DE.csv',low_memory=False).iloc[:-1, :]
columns = df['Production Type'].values
index = abbr_list
capacity = pd.DataFrame(index=index, columns=columns)
for abbr in abbr_list:
    df = pd.read_csv(f'../Data Sources/ENTSO-E/2018/Generation/Generation Capacity/{abbr}.csv', low_memory=False).iloc[:-1, :]
    for index, row in enumerate(df.values):
        capacity.loc[abbr][row[0]] = row[1]
capacity = capacity.replace(['n/e', np.nan], 0)
for column in capacity.columns.values:
    capacity[column] = pd.to_numeric(capacity[column])
del capacity['Marine']

# capacity_total = {}
# for abbr in abbr_list:
#     capacity_total[abbr] = {}
#     for item in entsoe_index:
#         capacity_total[abbr][item] = capacity.loc[abbr][item]


In [75]:
time_index = np.arange(8760, dtype=int)
em = pd.read_excel('../Data Sources/External sources.xlsx',
                   sheet_name='EUROSTAT', index_col=0)
eurostat_index = em.columns.values[:-3]

eurostat_total = {}
for abbr in abbr_list:
    eurostat_total[abbr] = {}
    for item in eurostat_index:
        eurostat_total[abbr][item] = em[item][abbr]*1000


In [72]:
countries = {'Norway': 'NO', 'Poland': 'PL', 'Portugal': 'PT',
             'Serbia': 'RS', 'Sweden': 'SE', 'Slovenia': 'SI', 'Slovakia': 'SK', 'United Kingdom': 'GB'}
abbr_list = list(countries.values())
abbr_list = ['NL']


In [76]:
capacity_total = {}
consolidated = {}
violations = {}

for abbr in abbr_list:
    generation_dic = {}
    df = pd.read_csv(f"../Data Sources/output/Sigma/Generation/{abbr}.csv").iloc[:, 1:]

    for column in df.columns.values:
        generation_dic[column] = df[column].sum()

    entsoe_index = []
    for key, value in generation_types.items():
        i = 0
        while i < len(value):
            entsoe_index.append(value[i])
            i += 1

    capacity_total[abbr] = {}
    
    for item in entsoe_index:
        if item not in df.columns.values:
            generation_dic[item] = 0.0
        capacity_total[abbr][item] = capacity.loc[abbr][item]

    model = pyo.ConcreteModel()

    model.eurostat_index = pyo.Set(initialize=eurostat_index)
    model.entsoe_index = pyo.Set(initialize=entsoe_index)

    model.alpha = pyo.Var(entsoe_index, bounds=(0.0, None))
    model.beta = pyo.Var(entsoe_index, bounds=(0.0, None))
    model.delta = pyo.Var(entsoe_index, bounds=(0.0, None))

    def gen_total(model,eurostat_generation):
        return sum((1+model.alpha[entsoe_generation])*generation_dic[entsoe_generation] + model.beta[entsoe_generation] - model.delta[entsoe_generation] for entsoe_generation in generation_types[eurostat_generation]) == eurostat_total[abbr][eurostat_generation]
    model.gen_total = pyo.Constraint(
        model.eurostat_index, rule=gen_total)


    def non_zero_rule(model, entsoe_generation):
        return (1+model.alpha[entsoe_generation])*generation_dic[entsoe_generation] + model.beta[entsoe_generation] - model.delta[entsoe_generation] >= 0
    model.non_zero_rule = pyo.Constraint(
        model.entsoe_index, rule=non_zero_rule)


    def ObjRule(model):
        return sum((model.alpha[entsoe_generation]*generation_dic[entsoe_generation])**2 + (model.beta[entsoe_generation] + model.delta[entsoe_generation])**2 for entsoe_generation in entsoe_index)
    model.obj = pyo.Objective(rule=ObjRule, sense=pyo.minimize)

    opt = SolverFactory("gurobi", solver_io="python")
    opt.solve(model)

    alpha = {}
    for fuel in entsoe_index:
        alpha[fuel] = model.alpha[fuel].value
        if alpha[fuel] is None:
            alpha[fuel] = 0

    consolidated[abbr] = {}
    for fuel in entsoe_index:
        consolidated[abbr][fuel] = (1 + alpha[fuel]) * \
            generation_dic[fuel] + \
            model.beta[fuel].value - model.delta[fuel].value
        if consolidated[abbr][fuel] < 0:
            consolidated[abbr][fuel] = 0.0

    generation_dic[abbr] = df
    
    violations[abbr] = []
    for column in df.columns.values:
        if 8760*capacity_total[abbr][column] <= consolidated[abbr][column]:
            violations[abbr].append(column)

    for item in violations[abbr]:
        if item in df.columns.values:
            del df[item]
     #######################
    solar_time_index = []
    if 'Solar' in df.columns.values:
        for index, values in enumerate(df.values):
            if df.loc[index, 'Solar'] < 0.1:
                solar_time_index.append(index)
    
    entsoe_index = df.columns.values
    model = pyo.ConcreteModel()

    model.entsoe_index = pyo.Set(initialize=entsoe_index)
    model.time_index = pyo.Set(initialize=time_index)

    model.alpha = pyo.Var(model.entsoe_index, bounds=(0.0, None))
    model.beta = pyo.Var(model.entsoe_index,model.time_index, bounds=(0.0, None))
    model.delta = pyo.Var(model.entsoe_index,model.time_index, bounds=(0.0, None))

    def gen_total_rule(model, entsoe_generation):
        return sum((1 + model.alpha[entsoe_generation])*generation_dic[abbr][entsoe_generation][time] + model.beta[entsoe_generation, time] - model.delta[entsoe_generation, time] for time in time_index) == consolidated[abbr][entsoe_generation]
    model.gen_total_rule = pyo.Constraint(
        model.entsoe_index, rule=gen_total_rule)

    def non_zero_rule(model, entsoe_generation, time):
        return (1 + model.alpha[entsoe_generation])*generation_dic[abbr][entsoe_generation][time] + model.beta[entsoe_generation, time] - model.delta[entsoe_generation, time] >= 0
    model.non_zero_rule = pyo.Constraint(
        model.entsoe_index, model.time_index, rule=non_zero_rule)

    def capacity_rule(model, entsoe_generation, time):
        return (1 + model.alpha[entsoe_generation])*generation_dic[abbr][entsoe_generation][time] + model.beta[entsoe_generation, time] - model.delta[entsoe_generation, time] <= capacity_total[abbr][entsoe_generation]
    model.capacity_rule = pyo.Constraint(
        model.entsoe_index, model.time_index, rule=capacity_rule)
    #############
    solar_index = ['Solar']
    model.solar_index = pyo.Set(initialize=solar_index)
    model.solar_time_index = pyo.Set(initialize=solar_time_index)

    if abbr != 'NL':
        def solar_rule(model, solar, time):
            return (1+model.alpha[solar])*generation_dic[abbr]['Solar'][time] + model.beta[solar, time] - model.delta[solar, time] == 0.0
        model.solar_rule = pyo.Constraint(
            model.solar_index, model.solar_time_index, rule=solar_rule)

    def ObjRule(model):
        return sum(model.alpha[entsoe_generation]*generation_dic[abbr][entsoe_generation][time]**2 + (model.beta[entsoe_generation, time] + model.delta[entsoe_generation, time])**2 for entsoe_generation in entsoe_index for time in time_index)
    model.obj = pyo.Objective(rule=ObjRule, sense=pyo.minimize)

    opt = SolverFactory("gurobi", solver_io="python")
    opt.solve(model)
    print(abbr, ': OPTIMIZATION COMPLETED')

    rows = []
    for time in time_index:
        row = []
        for entsoe_generation in entsoe_index:
            value = (1+ model.alpha[entsoe_generation].value)*generation_dic[abbr][entsoe_generation][time] + model.beta[entsoe_generation, time].value - model.delta[entsoe_generation, time].value
            if value<0:
                value = 0
            row.append(value)
        rows.append(row)
    dataframe = pd.DataFrame.from_records(rows)
    dataframe.columns = df.columns
    dataframe.to_csv(f"../Data Sources/output/External/Generation/{abbr}.csv")


NL : OPTIMIZATION COMPLETED


In [77]:
violations = {}
abbr = 'NL'
violations[abbr] = []
df = pd.read_csv(
    f"../Data Sources/output/Sigma/Generation/{abbr}.csv").iloc[:, 1:]
for column in df.columns.values:
    if 8760*capacity_total[abbr][column] <= consolidated[abbr][column]:
        violations[abbr].append(column)


In [78]:
violations


{'NL': ['Other']}